### Importações

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
from pycaret.classification import *
import os
from scripts.AtualizaTemporada import *
from scripts.TeoriaRetornos import *
from scripts.PreparaDatasetV2 import *
from scripts.AtualizaUltimosJogos import *
from scripts.BucketHandler import *
import glob

In [2]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)
    _jogos = _jogos.reset_index(drop=True)

    return _jogos

In [3]:
def met_bh_v4(jogos, data_var):
    df_bh_v4 = jogos.copy()

    df_bh_v4 = df_bh_v4[(df_bh_v4['FT_Odds_H'] >= 1.75) & (df_bh_v4['FT_Odds_H'] <= 2.25)]
    df_bh_v4.replace([np.inf, -np.inf], 0, inplace=True)

    if not df_bh_v4.empty:
        mdl_bh_v4 = load_model('ML/back_home/v4/v4_back_home', verbose=False)
        prev = predict_model(mdl_bh_v4, df_bh_v4)
        prev = prev[prev['prediction_label'] == 1]
        
        # Filtros pós modelo
        prev = prev[((prev['Saldo_Gols_A'] < -3) | (prev['Saldo_Gols_A'] > -1))]

        if not prev.empty:
            prev.to_csv(f'apostas_do_dia/{data_var}/back_home_v4_{data_var}.csv', index=False)
    else: print('Sem jogos')



In [4]:
def get_datas(lista):
  datas = []
  for item in lista:
    data = item.split("\\")[-1].split(".csv")[0]
    data = data.split("_")[-1]
    datas.append(data)
  return datas

In [5]:
itens = glob.glob('jogos_processados/prepara_dataset_v2/*.csv')
itens = get_datas(itens)
dias = itens[:-1]
dias

['2023-11-04', '2023-11-05', '2023-11-06', '2023-11-07']

In [6]:
for dia in dias:
    jogos = pd.read_csv(f'jogos_processados/prepara_dataset_v2/{dia}.csv')
    met_bh_v4(jogos, dia)
    met_bh_v5(jogos, dia)
    met_bh_v6(jogos, dia)